# Rating Product & Sorting Reviews in Amazon #


### Business Problem

One of the most significant problems in e-commerce is the accurate calculation of post-purchase ratings for products. Solving this problem means providing more customer satisfaction for the e-commerce site, highlighting products for sellers, and ensuring a seamless shopping experience for buyers. Another problem is the accurate sorting of product reviews. Since the prominence of misleading reviews can directly impact product sales, it can result in both financial losses and customer churn. After resolving these two core problems, the e-commerce site and sellers will increase their sales, while customers will be able to complete their purchase journey seamlessly.

*This dataset containing Amazon product data includes product categories along with various metadata. In the Electronics category, it includes user ratings and reviews for the product with the most reviews.*

## Variables ##

- reviewerID: User ID
- asin: Product ID
- reviewerName: User name
- helpful: Helpful Review Rating
- reviewText: 
- overall: Product Rating
- summary: Review Summary
- unixReviewTime: Review Date
- reviewTime: Review Date Raw
- day_diff: Number of Days Since Review
- helpful_yes: Number of Found Helpful Votes for the Review
- total_vote: The total number of votes given to the review, which includes both upvotes and downvotes.

First, we import the necessary libraries.

In [1]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In the shared dataset, users have given ratings and made reviews for a product. Our goal is to evaluate the given ratings by weighting them according to the date. Then we will compare the weighted score obtained with the initial average score.

Let's read and examine the dataset.

In [2]:
df = pd.read_csv("/kaggle/input/amazon-review/amazon_review.csv")

In [3]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [4]:
df.shape

(4915, 12)

Let's calculate the average rating of the product.

In [5]:
df["overall"].mean()

4.587589013224822

We are calculating the weighted average rating based on the date. For this, we are writing a function named 'time_based_weighted_average'. In this function, we are dividing the 'day_diff' variable, which represents the number of days since the review, into four quarters from the most recent to the oldest. Then, we assign weight values for each quarter.

In [6]:
df.loc[df["day_diff"] <= df["day_diff"].quantile(0.25), "overall"].mean()  # 4.69

df.loc[(df["day_diff"] > df["day_diff"].quantile(0.25)) & (df["day_diff"] <= df["day_diff"].
                                                           quantile(0.50)), "overall"].mean()  # 4.63

df.loc[(df["day_diff"] > df["day_diff"].quantile(0.50)) & (df["day_diff"] <= df["day_diff"].
                                                           quantile(0.75)), "overall"].mean()  # 4.57

df.loc[(df["day_diff"] > df["day_diff"].quantile(0.75)), "overall"].mean()  # 4.44

4.4462540716612375

Initially, when we examined the average ratings for each time period, we observed that the ratings given to the product increased as we got closer to the present day. Now, we can write our function for weight calculation

In [7]:
def time_based_weighted_average(dataframe, w1=28, w2=26, w3=24, w4=22):
    return dataframe.loc[dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.25), "overall"]. \
        mean() * w1 / 100 + \
        dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].
                       quantile(0.25)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(
            0.50)), "overall"].mean() * w2 / 100 + \
        dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].
                       quantile(0.50)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(
            0.75)), "overall"].mean() * w3 / 100 + \
        dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].
                       quantile(0.75)), "overall"].mean() * w4 / 100

When we run our function, we obtain the time-weighted average rating based on the date.

In [8]:
time_based_weighted_average(df)

4.595593165128118

In [9]:
df["overall"].mean()

4.587589013224822

We obtained a value of 4.58 by calculating only the overall variable, which is the average product rating. When we calculated the time-weighted rating average, we obtained a value of 4.59. It may not seem like a significant difference. However, for sellers to promote their products and for the shopping experience of customers, a difference of 0.01 is very valuable.

#### We will select the top 20 reviews to be displayed on the product's detail page. ####

First, let's generate the 'helpful_no' variable.
Note:
Total_vote is the total number of "up-down" given to a review.
Up" means "helpful.
There is no "helpful_no" variable in the dataset; it needs to be generated from the existing variables.

In [10]:
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

In [11]:
df = df[["reviewerName", "overall", "summary", "helpful_yes", "helpful_no", "total_vote", "reviewTime"]]

In [12]:
df.head()

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime
0,NaN,4.00000,Four Stars,0,0,0,2014-07-23
1,0mie,5.00000,MOAR SPACE!!!,0,0,0,2013-10-25
2,1K3,4.00000,nothing to really say....,0,0,0,2012-12-23
3,1m2,5.00000,Great buy at this price!!! *** UPDATE,0,0,0,2013-11-21
4,2&amp;1/2Men,5.00000,best deal around,0,0,0,2013-07-13


Let's calculate the 'score_pos_neg_diff,' 'score_average_rating,' and 'wilson_lower_bound' scores and add them to the data.

In [13]:
# score_pos_neg_diff:
def score_up_down_diff(up, down):
    return up - down

In [14]:
df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)

In [15]:
df.sort_values("score_pos_neg_diff", ascending=False).head(5)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349
3981,"R. Sutton, Jr. ""RWSynergy""",5.00000,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85


In [16]:
# score_average_rating:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [17]:
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)

In [18]:
df.sort_values("score_average_rating", ascending=False).head(5)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating
4277,S. Q.,5.00000,Perfect!!,1,0,1,2012-12-19,1,1.00000
2881,Lou Thomas,5.00000,Nexus One Loves This Card!,1,0,1,2012-01-10,1,1.00000
1073,C. Sanchez,5.00000,Tons of space for phone,1,0,1,2013-08-13,1,1.00000
445,"Apache ""Elizabeth""",4.00000,Amazon Great Prices,1,0,1,2013-12-18,1,1.00000
3923,Rock Your Roots,5.00000,What more to say?,1,0,1,2013-12-30,1,1.00000


In [19]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Calculate the Wilson Lower Bound Score

    - The lower bound of the confidence interval to be calculated for the Bernoulli parameter p is considered as the WLB score.
    - The calculated score is used for product ranking.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [20]:
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)

In [21]:
df.sort_values("wilson_lower_bound", ascending=False).head(5)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884,0.96634,0.95754
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351,0.94884,0.93652
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442,0.92562,0.91214
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349,0.85253,0.81858
4672,Twister,5.00000,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41,0.91837,0.80811


Let's select the top 20 reviews

In [22]:
df.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884,0.96634,0.95754
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351,0.94884,0.93652
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442,0.92562,0.91214
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349,0.85253,0.81858
4672,Twister,5.00000,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41,0.91837,0.80811
1835,goconfigure,5.00000,I own it,60,8,68,2014-02-28,52,0.88235,0.78465
3981,"R. Sutton, Jr. ""RWSynergy""",5.00000,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85,0.80576,0.73214
3807,R. Heisler,3.00000,"Good buy for the money but wait, I had an issue!",22,3,25,2013-02-27,19,0.88000,0.70044
4306,Stellar Eller,5.00000,Awesome Card!,51,14,65,2012-09-06,37,0.78462,0.67033
4596,"Tom Henriksen ""Doggy Diner""",1.00000,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55,0.75229,0.66359
